# Introduction

The PERCEIVE project seeks to proactively identify upcoming cybersecurity threats through textual similarity. Social network analysis, however, can be used to partition textual content, and hence serve as social-oriented word selection criteria, for defining corpus' documents. 

The subject of this notebook, the [Full Disclosure (FD) mailing list](http://seclists.org/fulldisclosure/) is a "public, vendor-neutral forum for detailed discussion of vulnerabilities and exploitation techniques, as well as tools, papers, news, and events of interest to the community."

## Problem Statement

Although cybersecurity email mailing lists provide a rich source to identify emerging concepts, it contains a large amount of content that is irrelevant to the project's purpose, including but not limited to:

 * conference invitations[[1](http://seclists.org/fulldisclosure/2017/Feb/6)]
 * vendor announcements[[2](http://seclists.org/fulldisclosure/2016/Dec/48)]
 * extensive conversations on security topics[[3](http://seclists.org/fulldisclosure/2004/Jul/1026)]
 * a significant amount of trolling[[4](http://seclists.org/fulldisclosure/2008/Apr/96)] 
 * and nonsense[[5](http://seclists.org/fulldisclosure/2009/Jul/289)] [[6](http://seclists.org/fulldisclosure/2004/Jul/796)].

As some of the irrelevant content can be strictly tied to it's producer, i.e., the **authors** of the **e-mail replies**, Social network analysis provides an interesting opportunity for the isolation of relevant discussion topics in order to **filter** non-related vulnerability content. 

## Network Definition

Earlier in this project, the FD email lists were developed into networks of edges and nodes, divided by year. The original csv files are available [online](https://mega.nz/#F!CUEByR5I!GY56GzTpYz68IlTqj4aQNQ!fR8jFLxL). The networks were imported into [Gephi](https://gephi.org/) to create a series of [visualization graphics](
https://mega.nz/#F!btdgFBID!hktkVrhZB5etOBBVLrgTrA).

In the FD graphs, nodes represent either documents (i.e., emails) or authors. Edges are directed, representing authorship and replies; edge weight indicates an increasing number of replies. In the Gephi graphics, blue nodes represent authors and red nodes represent documents. 


# Full Disclosure Exploration

## Considered Tools

### Gephi 

While Gephi has provided useful for data exploration, some difficulties arised. In particular, many of the core functions[[7](https://github.com/jaroslav-kuchar/Social-Network-Analysis/issues/2)] and plugins[[8](https://github.com/gephi/gephi/issues/1481)] once used for social network analysis are not compatible with current (0.9) software versions. Gephi specifications note that the 0.9.0 version (December 2015) "Removed [the] Clustering module"[[9](https://github.com/gephi/gephi/releases/tag/v0.9.0)] that these plugins relied upon.

### Python-igraph 

[Python-igraph](http://igraph.org/python/) (igraph hereafter) allows a user to import graphs in a variety of file formats, several of which match Gephi's export options. Brief tests demonstrated that the [GraphML](http://graphml.graphdrawing.org/) format was most successful for a workflow of transferring networks from Gephi to igraph. (Attempt to import [GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) and [Pajek](http://mrvar.fdv.uni-lj.si/pajek/) caused one instance of Python to hang; further investigation would be required to identify a cause.) The following examples use [GraphML files](https://mega.nz/#F!Dpdh2TjD!4Rd462mFXbdFn5Scs1WwUA) exported from Gephi.

## Network Definition 

In [6]:
from igraph import Graph, summary
from csv import DictReader

year = '2008'
# import edge list and vertex list as lists of dictionaries:
e = []
with open('data/author_threadID_edgelist_2008.csv') as csvfile:
    reader = DictReader(csvfile, dialect='excel')
    reader.fieldnames = [name.lower() for name in reader.fieldnames]  # igraph expects lowercase attribute names
    for row in reader:
        if year in row['target']:
            e.append(row)
v = []
with open('data/nodelist_aut_doc_2008.csv') as csvfile:
    reader = DictReader(csvfile, dialect='excel')
    reader.fieldnames = [name.lower() for name in reader.fieldnames]  # igraph expects lowercase attribute names
    for row in reader:
        v.append(row)

# build a graph from the lists; see http://igraph.org/python/doc/igraph.Graph-class.html#DictList

ml = Graph.DictList(vertices=v, edges=e, directed=True, vertex_name_attr='id')
# specify the 'id' attribute above because igraph anticipates a 'name' instead

ml['name'] = '2008 Full Disclosure network'  # provide a name for this graph

summary(ml)  # list properties and attributes

IGRAPH D-W- 9793 5591 -- 2008 Full Disclosure network
+ attr: name (g), color (v), id (v), label (v), label1 (v), label2 (v), nodetype (v), source (e), target (e), weight (e)


The summary command providing the output seen above is explained in the igraph documentation[[10](http://igraph.org/python/doc/igraph.summary'.GraphSummary-class.html)]. In this example, the four-character code "D-W-" indicates that the graph is directed and weighted. The graph has 9793 vertices (nodes) and 6463 edges. 

The attributes given in the summary ("name", "Color", etc.) are those for the graph (g), vertices (v), or edges (e). 

In [22]:
# show the id attribute of the first 2 vertices
print('ID list of first two vertices in network:')
print(ml.vs[0:2]['id'])


ID list of first two vertices in network:
['andur matrix <andurmatrix () gmail com>', 'Adam Muntner <adam.muntner () quietmove com>']


Nodes can either be **authors** as shown above, or **e-mail threads** (in which case their identifier is the year, month and relative id for the given month).

In [23]:
# identify vertices with the highest degree and betweenness
# (This sample is borrowed directly from the tutorial!)
print('\nIDs of vertices with highest degree:', ml.vs.select(_degree = ml.maxdegree())['id'])



IDs of vertices with highest degree: ['security () mandriva com']


It is often common that higher degree nodes (those who post several e-mail replies in e-mail threads) are advisories, and do not really communicate. We can clearly see that the author above, by his e-mail, is clearly a research group in the mailing list, and those are more related to advisories per our manual observations. 


## Filtering the nodes

The size of this network is too large for social network analysis to be useful across the entire graph. It is necessary to clean the 

In [3]:




# Q: how many nodes are there anyway?
print('total vertices:', len(ml.vs), 'including', len(ml.vs(nodetype_eq='author')), 'authors')

# 
# possible code for a visual overview of the in-degree of each author's neighboring nodes
# Note that the author's neighbors must be documents based on the way the graph was originally defined,
# connecting authors with (email) documents

print('list of all authors and their connected (document) vertices, with in-degree')
for node in ml.vs(nodetype_eq='author'):
    indegrees = [ml.degree(_, mode='in') for _ in ml.neighbors(node)]
    print()
    print('-' * 55)
    print(node['label'], '\n')
    print('  In-degree\t Label')
    print('  ---------\t -----')
    for _ in ml.neighbors(node):
        print(' ', ml.degree(_, mode='in'), '\t\t', ml.vs[_]['label'][0:60] )
    print('  MAX:', max(indegrees))
    print('  AVERAGE:', sum(indegrees) / max(len(indegrees), 1))

# Sample results of above:

#andur matrix <andurmatrix () gmail com> 
#
#  In-degree	 Label
#  ---------	 -----
#  1 		 Re: [OOT] Thesis for master degree
#  MAX: 1
#  AVERAGE: 1.0
#
#-------------------------------------------------------
#Adam Muntner <adam.muntner () quietmove com> 
#
#  In-degree	 Label
#  ---------	 -----
#  10 		 Re: [Professional IT Security Providers -	Exposed] QuietMove
#  1 		 Re: [Professional IT Security Providers -	Exposed] QuietMove
#
# ETC.



#print()
# now let's start filtering...
# first remove authors with neighbors mostly of in-degree 1
#print('Filtering out authors with neighbors mostly of in-degree 1...')
#for node in ml.vs:
#    if node['nodeType'] == 'author':
#        indegrees = [ml.degree(_, mode='in') for _ in ml.neighbors(node)]
#        if indegrees:  # a few authors have 0 neighbors... this is something to investigate later
#            if sum(indegrees) / max(len(indegrees), 1) < 1.2: #  there are examples where most of the indegrees are 1 but
#                                                     #  we have a random response, so let's use the mean
#                ml.delete_vertices(node) #  deleting vertices also deletes all of its edges


#print('total vertices:', len(ml.vs), 'including ', len(ml.vs(nodeType_eq='author')), 'authors')
#print()
# we still have too many communities, and a lot of isolated documents from 
# deleting the authors. let's remove all nodes with degree 0
#print('Filtering out documents or authors with degree 0...')
#for node in ml.vs:
#    if ml.degree(node) == 0: 
#        ml.delete_vertices(node)

#print('total vertices:', len(ml.vs), 'including ', len(ml.vs(nodeType_eq='author')), 'authors')
#print()




total vertices: 9793 including 2438 authors
list of all authors and their connected (document) vertices, with in-degree

-------------------------------------------------------
andur matrix <andurmatrix () gmail com> 

  In-degree	 Label
  ---------	 -----
  1 		 Re: [OOT] Thesis for master degree
  MAX: 1
  AVERAGE: 1.0

-------------------------------------------------------
Adam Muntner <adam.muntner () quietmove com> 

  In-degree	 Label
  ---------	 -----
  10 		 Re: [Professional IT Security Providers -	Exposed] QuietMove
  1 		 Re: [Professional IT Security Providers -	Exposed] QuietMove
  2 		 Re: [Professional IT Security Providers	-	Exposed] QuietMove
  5 		 Secreview re-review of quietmove ( F ---)
  3 		 Re: Fwd: Secreview re-review of quietmove ( F ---)
  18 		 EXPLOITS FOR SALE (AUCTION SITE)
  MAX: 18
  AVERAGE: 6.5

-------------------------------------------------------
Denzity <denzity () gmail com> 

  In-degree	 Label
  ---------	 -----
  2 		 Gmail XSS?
  MAX: 2
  

## Full Disclosure Visualization over Years 

The GraphML files from Gephi include the ids as _labels_, and so they are included by default when igraph creates visualizations. These may be eliminated by deleting the attribute.

Igraph handles visualization through _layouts_. Layouts are separate objects from the graphs themselves; multiple layouts can be created per graph.

Visual graphs are created via `plot` commands, however it demands too much time to render, and visualization is deferred to gephi and them edited on an image editor and assembled as a gif of 5s delay for easier visualization:

In [4]:
# import the GraphML files previously exported from Gephi
# delete label attributes to avoid visual clutter. In this example, the 'label'
# attribute is a duplicate of the 'ID' attribute so there is no need to save the label
#del ml.vs['label']

#layout = ml.layout_graphopt() # create a Fruchterman Reingold layout for the graph

#png = plot(ml, 'output/plot.png', layout=layout, bbox=(1000,1000), margin=100,
#           edge_curved=True)  # the bbox keyword argument defines the dimensions



The resulting graphic provides a visual overview of the FD 2016 network:

![An igraph-generated FD graph for 2016](https://github.com/sailuh/perceive/raw/master/Websites/Project/fulldisclosure_nooverlap_curved.gif)



We can observe that the network structure varies considerably across the years. This provides an opportunity to partition a network at a given year into several clusters and investigate if the visualized structure has any association to the textual content of a given **e-mail thread** discussion.

If such association exists, then we can leverage the **social network structure** in order to simplify the identification of emerging concepts through text alone. For instance, identifying a group of individuals have preference to certain subjects, or are spammers may become a trivial pre-processing stage before textual content is analyzed for emerging concepts.

We begin by considering 2 methods to more precisely define how to partition a network at any given year: 

 * Community Detection 
 * Betweenness Centrality

# Community Detection

Some of the community measures do not support directed graphs, as seen in the warning message below, but the direction can be removed to compare the results of igraph's community funtions.

In [33]:
com = ml.community_leading_eigenvector(weights='weight', clusters=3)

ValueError: attribute values must be numeric

In [318]:
ml.to_undirected(combine_edges='max') # eliminate the directionality

com = ml.community_leading_eigenvector(weights='weight', clusters=3)
print('clustering attempt, leading eigenvector:')
summary(com)


clustering attempt, leading eigenvector:
Clustering with 5889 elements and 2152 clusters


A more detailed look at the eigenvector clusters follows in two parts. First, a listing of the overall cluster contents:

In [319]:
print(len(com), 'clusters.')
print('maximum size:', len(max(com, key=len)))
print('minimum size:', len(min(com, key=len)))

print('\nSummary of first 5 clusters:')
for i in range(19):
    etc = ''
    if len(com[i]) > 5:
        etc += '(and ' + str(len(com[i]) - 5) + ' more)'
    print('[{}] has size of {}. Vertices: {} {}'.format(i, len(com[i]), com[i][0:5], etc))


2152 clusters.
maximum size: 3150
minimum size: 1

Summary of first 20 clusters:
[0] has size of 3150. Vertices: [0, 1, 2, 3, 4] (and 3145 more)
[1] has size of 256. Vertices: [34, 767, 1491, 1495, 1583] (and 251 more)
[2] has size of 6. Vertices: [35, 1492, 1533, 2170, 2361] (and 1 more)
[3] has size of 42. Vertices: [36, 2140, 2234, 2276, 4258] (and 37 more)
[4] has size of 7. Vertices: [42, 1409, 4361, 4392, 5049] (and 2 more)
[5] has size of 1. Vertices: [44] 
[6] has size of 1. Vertices: [72] 
[7] has size of 1. Vertices: [74] 
[8] has size of 7. Vertices: [75, 1585, 2564, 2679, 3032] (and 2 more)
[9] has size of 3. Vertices: [76, 78, 1537] 
[10] has size of 1. Vertices: [93] 
[11] has size of 2. Vertices: [98, 1567] 
[12] has size of 1. Vertices: [118] 
[13] has size of 1. Vertices: [119] 
[14] has size of 3. Vertices: [122, 1183, 1600] 
[15] has size of 4. Vertices: [128, 247, 1618, 1795] 
[16] has size of 5. Vertices: [133, 997, 2449, 3839, 3840] 
[17] has size of 78. Vertices:

Second, a report of the first 10 communities' node types and labels:

In [317]:
for i in range(10):
    print('COMMUNITY', i, '  ( total size:', len(com[i]), ')')
    for _ in com[i][0:10]:
        print('\t', ml.vs[_]['nodeType'], ':\t', ml.vs[_]['label'])
    if len(com[i]) > 10:
        print('\t... TRUNCATED LISTING')
    print()

COMMUNITY 0   ( total size: 3150 )
	 author :	 Adam Muntner <adam.muntner () quietmove com>
	 author :	 Denzity <denzity () gmail com>
	 author :	 Marcin Wielgoszewski <marcinw86 () gmail com>
	 author :	 Geo. <geoincidents () nls net>
	 author :	 coderman <coderman () gmail com>
	 author :	 reepex <reepex () gmail com>
	 author :	 Javor Ninov <drfrancky () securax org>
	 author :	 SilentRunner <silentrunner () hushmail com>
	 author :	 php0t <php0t () zorro hu>
	 author :	 Simon Smith <simon () snosoft com>
	... TRUNCATED LISTING

COMMUNITY 1   ( total size: 256 )
	 author :	 security () mandriva com
	 author :	 Jeroen Massar <jeroen () unfix org>
	 document :	 [ MDKSA-2007:001 ] - Update libmodplug packages	fix buffer overflow vulnerabilities
	 document :	 [ MDKSA-2007:002 ] - Updated kernel packages fix	multiple vulnerabilities and bugs
	 document :	 [ MDKSA-2007:003 ] - Updated avahi packages fix	DoS vulnerability
	 document :	 [ MDKSA-2007:004 ] - Updated geoip packages fix	geoipu

In [324]:
# There are too many clusters here; manual inspection suggests that we are mostly interested in the large cluster.
# This corresponds with the giant blob seen in the Gephi visualizations.

# Let's move from working with the entire graph to a subgraph including only this community

# TO DO: MAKE SURE WE ARE DEALING WITH THE LARGEST CLUSTER; HERE I KNOW IT IS CLUSTER ZERO


blob = ml.induced_subgraph(com[0])
blob['name'] = 'Subgraph for cluster 0 (blob)'
summary(blob)

IGRAPH U-W- 3150 4523 -- Subgraph for cluster 0 (blob)
+ attr: name (g), Label1 (v), Label2 (v), color (v), id (v), label (v), nodeType (v), Edge Label (e), id (e), weight (e)


In [325]:
# Now working with the blob, we can re-attempt clustering and bc!

com = blob.community_leading_eigenvector(weights='weight', clusters=8)
print('clustering attempt, leading eigenvector:')
summary(com)



clustering attempt, leading eigenvector:
Clustering with 3150 elements and 6 clusters


# Betweenness Centrality

Igraph betweenness measures, like the clustering algorithms, require undirected graphs. Betweenness is returned as a simple list of values.

In [326]:
bc = blob.betweenness()
bc.sort(reverse=True);

In [327]:
max_bc = max(bc)
bc_normalized = [x / max_bc for x in bc]

for idx,val in enumerate(bc_normalized[0:17]):
    print('Node',idx,':',"{:.0%}".format(bc_normalized[idx]))

Node 0 : 100%
Node 1 : 48%
Node 2 : 39%
Node 3 : 37%
Node 4 : 36%
Node 5 : 35%
Node 6 : 35%
Node 7 : 35%
Node 8 : 34%
Node 9 : 33%
Node 10 : 32%
Node 11 : 31%
Node 12 : 19%
Node 13 : 17%
Node 14 : 17%
Node 15 : 16%
Node 16 : 15%


We can observe that the betweeness centrality quickly drops past the 16th node, which may indicate nodes that often discuss different subjects for contributing e-mail discussions for different "community of threads". 

# Exporting and utilizing analyzed graph data

Igraph allows users to export graph files for use in other software such as Gephi.

In [331]:
# add betweenness centrality, pagerank, and clustering info to original graph
bc = blob.betweenness()
for idx, node in enumerate(blob.vs):
    blob.vs[idx]['blob_betweenness'] = bc[idx]
    
pr = blob.pagerank(directed=False)
for idx, node in enumerate(blob.vs):
    blob.vs[idx]['blob_pagerank'] = pr[idx]
for idx, c in enumerate(com):
    for _ in c:
        blob.vs[_]['blob_cluster'] = idx

blob.save('data/blob2008_out.graphml', format='graphml') # export graph

# now let's try each cluster separately

subs = {}
for idx, c in enumerate(com):
    subs[idx] = blob.induced_subgraph(com[idx])
    subs[idx]['name'] = '2008 blob subgraph ' + str(idx) + ' of ' + str(len(com))
    # rerun the centrality, clustering, pagerank analyses
    # (This could certainly use some functions for DRY principle)
    bc = subs[idx].betweenness()
    for idy, node in enumerate(subs[idx].vs):
        subs[idx].vs[idy]['local_betweenness'] = bc[idy]
    pr = subs[idx].pagerank()
    for idy, node in enumerate(subs[idx].vs):
        subs[idx].vs[idy]['local_pagerank'] = pr[idy]
    subcom = subs[idx].community_leading_eigenvector(weights='weight', clusters=5)
    for idy, c in enumerate(subcom):
        for _ in c:
            subs[idx].vs[_]['local_cluster'] = idy
    filename = 'data/' + subs[idx]['name'].replace(' ', '_') + '_out.graphml'
    subs[idx].save(filename, format='graphml')


In [332]:
from igraph import layout, plot

for _ in subs:
    # delete label attributes to avoid visual clutter. In this example, the 'label'
    del subs[_].vs['label']
    layout = subs[_].layout_graphopt() # create a Fruchterman Reingold layout for the graph
    filename = 'output/subgraph' + str(_) + '_' + str(len(subs[_].vs)) + 'n.png'
    png = plot(subs[_], filename, layout=layout, bbox=(1000,1000), margin=100, edge_curved=True)
    print(filename, 'generated.')


output/subgraph0_121n.png generated.
output/subgraph1_2532n.png generated.
output/subgraph2_168n.png generated.
output/subgraph3_151n.png generated.
output/subgraph4_34n.png generated.
output/subgraph5_144n.png generated.


After opening this 2008 graph in Gephi, we can manipulate colors and sizes of nodes to reflect attributes identified in igraph. Adjusting colors based on clustering, and node size based on betweenness centrality, provides a visual overview of the newly-identified features:

![FD2008 graph from Gephi](data/2008_clusters_betweenness.png)

We can observe thousands of small nodes with minimal betweenness value (over 8000 nodes in the graph have a betweenness of zero), and a handful of large nodes with higher values. The clusters are colored via cluster partition; one large cluster colored purple accounts for more than a third of the overall graph.

Manual analysis of the large (high-betweenness) clusters indicates that these nodes represent users[[1](https://www.bing.com/search?q=site%3Aseclists.org+Valdis.Kletnieks+%28%29+vt+edu)][[2](https://www.bing.com/search?q=site%3Aseclists.org+3APA3A+%3C3APA3A+%28%29+SECURITY+NNOV+RU)] who are extremely active on the mailing list in multiple threads, partially confirming our hypothesis.



In [ ]:
# TO DO: manual analysis of clusters;
#        code to automate the process for multiple years;
#        hypotheses for filtering/deleting clusters or nodes by attribute